In [2]:
import numpy as np
import random

# pretty print for np.array 
# from https://stackoverflow.com/questions/53126305/pretty-printing-numpy-ndarrays-using-unicode-characters/53164538#53164538
def pretty_print(A):
    if A.ndim==1:
        print(A)
    else:
        w = max([len(str(s)) for s in A]) 
        print(u'\u250c' + u'\u2500' * w + u'\u2510') 
        for AA in A:
            print(' ', end='')
            print('[', end='')
            for i, AAA in enumerate(AA[:-1]):
                w1 = max([len(str(s)) for s in A[:, i]])
                print(str(AAA) + ' ' * (w1  - len(str(AAA)) + 1), end='')
            w1 = max([len(str(s)) for s in A[:, -1]])
            print(str(AA[-1]) + ' ' * (w1 - len(str(AA[-1]))), end='')
            print(']')
        print(u'\u2514'+u'\u2500' * w + u'\u2518')  

def compare(left: set, right: set):
    if len(left) == len(right):
        for i in range(len(left)):
            if left[i] != right[i]:
                return left[i] < right[i]
    return left < right


### Generate $\mathcal{A}_n^2$

> Let $|\cdot|$ denotes cardinality and $\Delta$ denote symmetric different

In [3]:
def Delta(left: set, right: set):
    return left.symmetric_difference(right)

def Cardi(x: set):
    return len(x)

def legal(alpha_i: set, alpha_i_1: set):
    return Cardi(alpha_i) <= Cardi(alpha_i_1) and len(Delta(alpha_i, alpha_i_1)) <= 2

> Let $n=2^m$, $\Omega$ be a set of $m$ element such that $|\alpha_i| \leq |\alpha_{i+1}|$ and $|\alpha_i \Delta \alpha_{i+1}| \leq 2$  
> Let $\alpha_0=\{\varnothing\}$, now genereate $\Omega$

In [4]:
from itertools import chain, combinations

# param
m = 4
# generate \Omega

def create_Omega(n):
    s = list(range(1, n + 1))
    container = chain.from_iterable(combinations(s, r) for r in range(len(s)+1))
    map = {i: [] for i in range(1, len(s) + 1)}
    for item in container:
        if len(item) != 0:
            map[len(item)].append(set(item))
    res = [set()]
    for key, value in map.items():
        if key % 2 == 0:
            value.reverse()
        res.extend(value)
    return res

omega = create_Omega(m)

for i in range(m - 1):
    assert(legal(omega[i], omega[i + 1]))
assert(len(omega) == 2**m)

omega

[set(),
 {1},
 {2},
 {3},
 {4},
 {3, 4},
 {2, 4},
 {2, 3},
 {1, 4},
 {1, 3},
 {1, 2},
 {1, 2, 3},
 {1, 2, 4},
 {1, 3, 4},
 {2, 3, 4},
 {1, 2, 3, 4}]

> $$a_{ij} = \begin{cases}
    -1,\;\alpha_j\bigcap(\alpha_{i-1}\bigcup\alpha_i)=\alpha_{i-1}\Delta\alpha_{i}\;and\;|\alpha_{i-1}\Delta\alpha_{i}|=2 \\
    (-1)^{|\alpha_{i-1}\bigcap\alpha_j| + 1},\;\alpha_{j}\bigcap(\alpha_{i-1}\bigcup\alpha_{i})\neq\varnothing\;but\;does\;not\;meet\;the\;condition\;above \\
    1,\;\alpha_j\bigcap(\alpha_{i-1}\bigcup\alpha_i)=\varnothing \\
\end{cases}
$$

In [11]:
def query_element(i: int, j: int, omega: list) -> int:
    alpha_j = omega[j]
    alpha_i_1 = omega[i - 1]
    alpha_i = omega[i]

    if alpha_j.intersection(alpha_i_1.union(alpha_i)) == Delta(alpha_i_1, alpha_i) \
        and Cardi(Delta(alpha_i_1, alpha_i)) == 2:
        return -1
    elif Cardi(alpha_j.intersection(alpha_i_1.union(alpha_i))) != 0:
        return (-1)**(Cardi(alpha_i_1.intersection(alpha_j)) + 1)
    elif Cardi(alpha_j.intersection(alpha_i_1.union(alpha_i))) == 0:
        return 1
    else:
        raise ValueError("Undefined behavior!")
    
A_mat = np.zeros((2**m, 2**m))
for i in range(A_mat.shape[0]):
    for j in range(A_mat.shape[1]):
        A_mat[i, j] = query_element(i, j, omega)

pretty_print(A_mat.astype(int))

┌─────────────────────────────────────────────────┐
 [1 1  1  1  1  -1 -1 -1 -1 -1 -1 1  1  1  1  -1]
 [1 -1 1  1  1  1  1  1  -1 -1 -1 -1 -1 -1 1  -1]
 [1 1  -1 1  1  1  -1 -1 1  1  -1 -1 -1 1  -1 -1]
 [1 1  1  -1 1  -1 1  -1 1  -1 1  -1 1  -1 -1 -1]
 [1 1  1  1  -1 -1 -1 1  -1 1  1  1  -1 -1 -1 -1]
 [1 1  1  -1 1  1  1  -1 1  -1 1  -1 1  1  1  1 ]
 [1 1  -1 1  1  -1 1  -1 1  1  -1 -1 1  -1 -1 -1]
 [1 1  1  -1 1  -1 -1 1  1  -1 1  1  -1 -1 -1 -1]
 [1 -1 1  1  -1 1  1  -1 -1 1  1  -1 1  1  -1 -1]
 [1 1  1  -1 1  -1 1  -1 -1 1  1  1  -1 -1 -1 -1]
 [1 1  -1 1  1  1  -1 -1 1  -1 1  -1 1  -1 -1 -1]
 [1 1  1  -1 1  -1 1  1  1  1  -1 -1 -1 1  1  -1]
 [1 1  1  1  -1 -1 1  -1 1  -1 -1 1  -1 -1 -1 1 ]
 [1 1  1  -1 1  1  -1 -1 -1 1  -1 -1 1  -1 -1 1 ]
 [1 1  -1 1  1  -1 1  1  -1 -1 -1 -1 -1 1  -1 1 ]
 [1 -1 1  1  1  -1 -1 -1 1  1  1  -1 -1 -1 1  1 ]
└─────────────────────────────────────────────────┘


### Generate $\mathcal{A}_{n - 1}^1$

> Let $B\in\mathcal{A}_{n - 1}^1$, $\Phi(B)\in\mathcal{A}_{n}^2\;$:
> $$ \Phi(B) = \left(\begin{array}{cc} 
1 & 1_{n-1}\\
-1^T_{n-1} & 2B-J_{n-1}
\end{array}\right)
> $$
> Thus: 
> $$ \Phi(B)^{-1} = \left(\begin{array}{cc} 
1-\frac{1}{2}1_{n-1}B^{-1}1^T_{n-1} & -\frac{1}{2}1_{n-1}B^{-1}\\
\frac{1}{2}B^{-1}1^T_{n-1} & \frac{1}{2}B^{-1}
\end{array}\right)
> $$

In [27]:
def rev_Phi(B):
    B_mat = np.matrix(B)
    B_mat_I = B_mat.I
    n = B.shape[0]
    ones = np.ones((1, n))
    left_top = 1 - 0.5 * ones * B_mat_I * ones.T
    print(left_top)
    right_top = -0.5 * ones * B_mat_I
    left_btn = 0.5 * B_mat_I * ones.T
    right_btn = 0.5 * B_mat_I
    top = np.concatenate((left_top, right_top), axis=1)
    btn = np.concatenate((left_btn, right_btn), axis=1)
    mat = np.concatenate((top, btn), axis=0)

    return mat

pretty_print(rev_Phi(A_mat))

[[0.5]]
┌───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
 [[[ 0.5         0.11570248 -0.80991736 -1.54545455 -0.47933884 -1.59090909
  -2.46280992 -1.99173554 -0.87603306  0.61570248  0.87603306  3.2231405
   2.46280992  0.97520661  0.47933884  0.31404959  0.19421488]]]
 [[[ 0.5         0.05785124 -0.40495868 -1.02272727 -0.98966942 -0.79545455
  -0.73140496 -0.49586777  0.06198347  0.55785124  0.43801653  1.61157025
   1.23140496  0.48760331  0.23966942  0.15702479  0.09710744]]]
 [[[ 9.02056208e-17 -1.44628099e-02  1.01239669e-01  3.18181818e-01
   3.09917355e-01  3.86363636e-01  3.07851240e-01  1.23966942e-01
  -1.40495868e-01 -2.64462810e-01 -1.09504132e-01 -4.02892562e-01
  -3.07851240e-01 -1.21900826e-01 -5.99173554e-02 -3.92561983e-02
  -8.677685